In [1]:
# Import libraries necessary for this project

# Pandas, and NumPy: 
import numpy as np
import pandas as pd
from time import time
import math

# Plotly: 
import chart_studio.plotly as py
import plotly
import plotly.graph_objs as go
from plotly import tools
import csv
import folium
import matplotlib.pyplot as pl
import matplotlib.patches as mpatches

# sklearn: 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

# folium: 
from folium import plugins
from folium.plugins import HeatMap

# dython: 
from dython.nominal import associations

import colorlover as cl
from IPython.display import HTML
from IPython.display import display

# Pretty display for notebooks
%matplotlib inline

In [12]:
# Load datasets
# 2015
accidents_data_2015 = pd.read_csv("Desktop/TrafficIncidents/GBR_2015_accidents.csv", parse_dates=['Date'], date_parser = pd.to_datetime)
vehicles_data_2015 = pd.read_csv("Desktop/TrafficIncidents/GBR_2015_vehicles.csv")
casualties_data_2015 = pd.read_csv("Desktop/TrafficIncidents/GBR_2015_casualties.csv")
# 2016
accidents_data_2016 = pd.read_csv("Desktop/TrafficIncidents/GBR_2016_accidents.csv", parse_dates=['Date'], date_parser = pd.to_datetime)
vehicles_data_2016 = pd.read_csv("Desktop/TrafficIncidents/GBR_2016_vehicles.csv")
casualties_data_2016 = pd.read_csv("Desktop/TrafficIncidents/GBR_2016_casualties.csv")
# 2017
accidents_data_2017 = pd.read_csv("Desktop/TrafficIncidents/GBR_2017_accidents.csv", parse_dates=['Date'], date_parser = pd.to_datetime)
vehicles_data_2017 = pd.read_csv("Desktop/TrafficIncidents/GBR_2017_vehicles.csv")
casualties_data_2017 = pd.read_csv("Desktop/TrafficIncidents/GBR_2017_casualties.csv")
# 2018
accidents_data_2018 = pd.read_csv("Desktop/TrafficIncidents/GBR_2018_accidents.csv", parse_dates=['Date'], date_parser = pd.to_datetime)
vehicles_data_2018 = pd.read_csv("Desktop/TrafficIncidents/GBR_2018_vehicles.csv")
casualties_data_2018 = pd.read_csv("Desktop/TrafficIncidents/GBR_2018_casualties.csv")
# 2019
accidents_data_2019 = pd.read_csv("Desktop/TrafficIncidents/GBR_2019_accidents.csv", parse_dates=['Date'], date_parser = pd.to_datetime)
vehicles_data_2019 = pd.read_csv("Desktop/TrafficIncidents/GBR_2019_vehicles.csv")
casualties_data_2019 = pd.read_csv("Desktop/TrafficIncidents/GBR_2019_casualties.csv")

/Users/nataliergravett/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning:

Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.



In [13]:
# concat all datasets into one: 
accidents_data = pd.concat([accidents_data_2015, accidents_data_2016, accidents_data_2017, accidents_data_2018, accidents_data_2019], ignore_index=True)
vehicles_data = pd.concat([vehicles_data_2015, vehicles_data_2016, vehicles_data_2017, vehicles_data_2018, vehicles_data_2019], ignore_index=True)
casualties_data = pd.concat([casualties_data_2015, casualties_data_2016, casualties_data_2017, casualties_data_2018, casualties_data_2019], ignore_index=True)

In [14]:
display(accidents_data.head())

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,201501BS70001,525130.0,180050.0,-0.198465,51.505538,1,3,1,1,2015-12-01,...,0,0,4,1,1,0,0,1,1,E01002825
1,201501BS70002,526530.0,178560.0,-0.178838,51.491836,1,3,1,1,2015-12-01,...,0,0,1,1,1,0,0,1,1,E01002820
2,201501BS70004,524610.0,181080.0,-0.205590,51.514910,1,3,1,1,2015-12-01,...,0,1,4,2,2,0,0,1,1,E01002833
3,201501BS70005,524420.0,181080.0,-0.208327,51.514952,1,3,1,1,2015-01-13,...,0,0,1,1,2,0,0,1,2,E01002874
4,201501BS70008,524630.0,179040.0,-0.206022,51.496572,1,2,2,1,2015-09-01,...,0,5,1,2,2,0,0,1,2,E01002814


In [7]:
accidents_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390178 entries, 0 to 390177
Data columns (total 32 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   Accident_Index                               390178 non-null  object        
 1   Location_Easting_OSGR                        390097 non-null  float64       
 2   Location_Northing_OSGR                       390097 non-null  float64       
 3   Longitude                                    390087 non-null  float64       
 4   Latitude                                     390087 non-null  float64       
 5   Police_Force                                 390178 non-null  int64         
 6   Accident_Severity                            390178 non-null  int64         
 7   Number_of_Vehicles                           390178 non-null  int64         
 8   Number_of_Casualties                         390178 non-null  in

In [15]:
display(vehicles_data.head())

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Vehicle_IMD_Decile
0,201506E098757,2,9,0,18,0,8,0,0,0,...,6,1,45,7,1794,1,11,-1,1,-1
1,201506E098766,1,9,0,9,0,8,0,0,0,...,6,2,25,5,1582,2,1,-1,-1,-1
2,201506E098766,2,9,0,18,0,8,0,0,0,...,6,1,51,8,-1,-1,-1,-1,1,-1
3,201506E098777,1,20,0,4,0,0,0,0,0,...,1,1,50,8,4462,2,1,-1,1,-1
4,201506E098780,1,9,0,15,0,1,0,0,0,...,6,1,27,6,1598,2,-1,-1,1,-1


In [16]:
vehicles_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192061 entries, 0 to 1192060
Data columns (total 23 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   Accident_Index                    1192061 non-null  object
 1   Vehicle_Reference                 1192061 non-null  int64 
 2   Vehicle_Type                      1192061 non-null  int64 
 3   Towing_and_Articulation           1192061 non-null  int64 
 4   Vehicle_Manoeuvre                 1192061 non-null  int64 
 5   Vehicle_Location-Restricted_Lane  1192061 non-null  int64 
 6   Junction_Location                 1192061 non-null  int64 
 7   Skidding_and_Overturning          1192061 non-null  int64 
 8   Hit_Object_in_Carriageway         1192061 non-null  int64 
 9   Vehicle_Leaving_Carriageway       1192061 non-null  int64 
 10  Hit_Object_off_Carriageway        1192061 non-null  int64 
 11  1st_Point_of_Impact               1192061 non-null

In [17]:
display(casualties_data.head())

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,201597UA71710,2,1,1,2,75,10,3,0,0,0,0,0,9,3,-1
1,201597UA71810,2,1,2,2,63,9,2,0,0,0,4,0,11,3,-1
2,201597UA71810,2,2,2,2,75,10,2,0,0,0,4,0,11,1,-1
3,201597UA71810,2,3,2,1,78,11,2,0,0,0,4,0,11,1,-1
4,201597UA71810,2,4,2,1,67,10,2,0,0,0,4,0,11,1,-1


In [18]:
casualties_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852321 entries, 0 to 852320
Data columns (total 16 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Accident_Index                      852321 non-null  object
 1   Vehicle_Reference                   852321 non-null  int64 
 2   Casualty_Reference                  852321 non-null  int64 
 3   Casualty_Class                      852321 non-null  int64 
 4   Sex_of_Casualty                     852321 non-null  int64 
 5   Age_of_Casualty                     852321 non-null  int64 
 6   Age_Band_of_Casualty                852321 non-null  int64 
 7   Casualty_Severity                   852321 non-null  int64 
 8   Pedestrian_Location                 852321 non-null  int64 
 9   Pedestrian_Movement                 852321 non-null  int64 
 10  Car_Passenger                       852321 non-null  int64 
 11  Bus_or_Coach_Passenger              852

In [19]:
lookup_mapping = pd.read_csv('Desktop/TrafficIncidents/variable_lookup.csv', header=None, index_col=0, squeeze=True).to_dict()

In [20]:
lookup_mapping

{1: {'VARIABLES': nan,
  nan: nan,
  'Accident Circumstances': 'Vehicle',
  'Accident Index': 'Accident Index',
  'Police Force': 'Vehicle Reference',
  'Accident Severity': 'Vehicle Type',
  'Number of Vehicles': 'Towing and Articulation',
  'Number of Casualties': 'Vehicle Manoeuvre',
  'Date (DD/MM/YYYY)': 'Vehicle Location-Restricted Lane',
  'Day of Week': 'Junction Location',
  'Time (HH:MM)': 'Skidding and Overturning',
  'Location Easting OSGR (Null if not known)': 'Hit Object in Carriageway',
  'Location Northing OSGR (Null if not known)': 'Vehicle Leaving Carriageway',
  'Longitude (Null if not known)': 'Hit Object off Carriageway',
  'Latitude (Null if not known)': '1st Point of Impact',
  'Local Authority (District)': 'Was Vehicle Left Hand Drive',
  'Local Authority (Highway Authority - ONS code)': 'Journey Purpose of Driver',
  '1st Road Class': 'Sex of Driver',
  '1st Road Number': 'Age of Driver',
  'Road Type': 'Age Band of Driver',
  'Speed limit': 'Engine Capacity',


In [21]:
# for all features, (-1) represent a missing data point, to better handle it with pandas, it will be replaced to NaN: 
accidents_data.replace(-1, 'NaN', inplace=True)
casualties_data.replace(-1, 'NaN', inplace=True)
vehicles_data.replace(-1, 'NaN', inplace=True)

In [23]:
# time series features: 
accidents_data['Year'] = accidents_data.Date.dt.year 
accidents_data['Month_number'] = accidents_data.Date.dt.month
accidents_data['Month'] = accidents_data.Date.dt.month_name()
accidents_data['Hour'] = accidents_data.apply(lambda x: str(x.Time).split(':')[0], axis=1)

In [24]:
# merge vehicle data using key (Accident_Index)
accidents_data = pd.merge(accidents_data,vehicles_data[['Accident_Index','Vehicle_Type','Sex_of_Driver', 'Age_of_Driver', 'Age_Band_of_Driver','Vehicle_Manoeuvre']],on='Accident_Index', how='left')

In [25]:
# Data information after merging: 
print ('accidents_data before cleanup:', accidents_data.shape[0])
print ('casualties_data before cleanup:', casualties_data.shape[0])
print ('vehicles_data before cleanup:', vehicles_data.shape[0])

accidents_data before cleanup: 945271
casualties_data before cleanup: 852321
vehicles_data before cleanup: 1192061


In [27]:
# remove dublicate 
accidents_data.drop_duplicates(subset='Accident_Index', keep='first', inplace=True)

print ('accidents_data drop duplicates:', accidents_data.shape[0])
print ('casualties_data drop duplicates:', casualties_data.shape[0])
print ('vehicles_data drop duplicates:', vehicles_data.shape[0])

accidents_data drop duplicates: 529295
casualties_data drop duplicates: 852321
vehicles_data drop duplicates: 1192061


In [28]:
accidents_data_cleanup = ['Longitude','Time','Speed_limit', 'Junction_Detail', 'Light_Conditions', 'Road_Type',
                          'Did_Police_Officer_Attend_Scene_of_Accident','Weather_Conditions', 
                          'Road_Surface_Conditions', 'Special_Conditions_at_Site','Carriageway_Hazards', 
                          'Urban_or_Rural_Area', 'Accident_Severity', '1st_Road_Class', 
                          'Pedestrian_Crossing-Human_Control', 
                          'Pedestrian_Crossing-Physical_Facilities', 
                          'Police_Force','Vehicle_Type', 'Sex_of_Driver', 'Age_of_Driver', 
                          'Vehicle_Manoeuvre']

accidents_data_cleanup_outliers = ['Sex_of_Driver', 'Age_Band_of_Driver']
outliers_list = ['6 - 10', '0 - 5', 'Not known']

In [29]:
# method to clean spesific features, and remove linked data points in other datasets: 
def clean_data (cleanup_features, df, df2, df3): 
    for feature in cleanup_features: 
        accidents_data_list = df[(df[feature].isnull()) | (df[feature] == 'NaN')].Accident_Index.tolist()
        df = df[~df.Accident_Index.isin(accidents_data_list)]
        df2 = df2[~df2.Accident_Index.isin(accidents_data_list)]
        df3 = df3[~df3.Accident_Index.isin(accidents_data_list)]
    return df, df2, df3

In [31]:
# method to remove outliers from spesific features, and remove linked data points in other datasets: 
def clean_outliers (cleanup_outliers, outliers_list, df, df2, df3): 
    for feature in cleanup_outliers: 
        accidents_data_list = df[(df[feature].isin (outliers_list))].Accident_Index.tolist()
        df = df[~df.Accident_Index.isin(accidents_data_list)]
        df2 = df2[~df2.Accident_Index.isin(accidents_data_list)]
        df3 = df3[~df3.Accident_Index.isin(accidents_data_list)]
    return df, df2, df3

In [32]:
accidents_data.isnull().sum(axis = 0)

Accident_Index                                     0
Location_Easting_OSGR                            108
Location_Northing_OSGR                           108
Longitude                                        118
Latitude                                         118
Police_Force                                       0
Accident_Severity                                  0
Number_of_Vehicles                                 0
Number_of_Casualties                               0
Date                                               0
Day_of_Week                                        0
Time                                              36
Local_Authority_(District)                         0
Local_Authority_(Highway)                          0
1st_Road_Class                                     0
1st_Road_Number                                    0
Road_Type                                          0
Speed_limit                                       37
Junction_Detail                               

In [34]:
# clean data to remove rows with missing values/outliers and their linked vehicles/casualties data
accidents_data, casualties_data, vehicles_data = clean_data(accidents_data_cleanup, accidents_data, casualties_data, vehicles_data)
accidents_data, casualties_data, vehicles_data = clean_outliers(accidents_data_cleanup_outliers, outliers_list, accidents_data, casualties_data, vehicles_data)

In [35]:
print ('accidents_data final cleanup:', accidents_data.shape[0])
print ('casualties_data final cleanup:', casualties_data.shape[0])
print ('vehicles_data final cleanup:', vehicles_data.shape[0])

accidents_data final cleanup: 429239
casualties_data final cleanup: 773502
vehicles_data final cleanup: 1079601


In [36]:
accidents_data.isnull().sum(axis = 0)/accidents_data.shape[0]*100

Accident_Index                                 0.000000
Location_Easting_OSGR                          0.000000
Location_Northing_OSGR                         0.000000
Longitude                                      0.000000
Latitude                                       0.000000
Police_Force                                   0.000000
Accident_Severity                              0.000000
Number_of_Vehicles                             0.000000
Number_of_Casualties                           0.000000
Date                                           0.000000
Day_of_Week                                    0.000000
Time                                           0.000000
Local_Authority_(District)                     0.000000
Local_Authority_(Highway)                      0.000000
1st_Road_Class                                 0.000000
1st_Road_Number                                0.000000
Road_Type                                      0.000000
Speed_limit                                    0

In [37]:
accidents_data.drop(["LSOA_of_Accident_Location", "2nd_Road_Class", "Junction_Control"], axis=1, inplace=True)

In [38]:
casualties_data.isnull().sum(axis = 0)/casualties_data.shape[0]*100

Accident_Index                        0.0
Vehicle_Reference                     0.0
Casualty_Reference                    0.0
Casualty_Class                        0.0
Sex_of_Casualty                       0.0
Age_of_Casualty                       0.0
Age_Band_of_Casualty                  0.0
Casualty_Severity                     0.0
Pedestrian_Location                   0.0
Pedestrian_Movement                   0.0
Car_Passenger                         0.0
Bus_or_Coach_Passenger                0.0
Pedestrian_Road_Maintenance_Worker    0.0
Casualty_Type                         0.0
Casualty_Home_Area_Type               0.0
Casualty_IMD_Decile                   0.0
dtype: float64

In [39]:
casualties_data.drop(["Casualty_IMD_Decile", "Pedestrian_Road_Maintenance_Worker"], axis=1, inplace=True)

In [40]:
vehicles_data.isnull().sum(axis = 0)/vehicles_data.shape[0]*100

Accident_Index                      0.0
Vehicle_Reference                   0.0
Vehicle_Type                        0.0
Towing_and_Articulation             0.0
Vehicle_Manoeuvre                   0.0
Vehicle_Location-Restricted_Lane    0.0
Junction_Location                   0.0
Skidding_and_Overturning            0.0
Hit_Object_in_Carriageway           0.0
Vehicle_Leaving_Carriageway         0.0
Hit_Object_off_Carriageway          0.0
1st_Point_of_Impact                 0.0
Was_Vehicle_Left_Hand_Drive?        0.0
Journey_Purpose_of_Driver           0.0
Sex_of_Driver                       0.0
Age_of_Driver                       0.0
Age_Band_of_Driver                  0.0
Engine_Capacity_(CC)                0.0
Propulsion_Code                     0.0
Age_of_Vehicle                      0.0
Driver_IMD_Decile                   0.0
Driver_Home_Area_Type               0.0
Vehicle_IMD_Decile                  0.0
dtype: float64

In [41]:
vehicles_data.drop(["Driver_IMD_Decile", "Vehicle_IMD_Decile"], axis=1, inplace=True)

In [42]:
accidents_data.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Did_Police_Officer_Attend_Scene_of_Accident,Year,Month_number,Month,Hour,Vehicle_Type,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Vehicle_Manoeuvre
2,201501BS70004,524610.0,181080.0,-0.205590,51.514910,1,3,1,1,2015-12-01,...,1,2015,12,December,18,9,1,30,6,9
4,201501BS70008,524630.0,179040.0,-0.206022,51.496572,1,2,2,1,2015-09-01,...,2,2015,9,September,07,1,1,48,8,18
6,201501BS70009,525480.0,179530.0,-0.193610,51.500788,1,3,2,1,2015-01-15,...,1,2015,1,January,09,3,1,37,7,18
8,201501BS70010,526890.0,178940.0,-0.173519,51.495171,1,3,2,1,2015-01-15,...,1,2015,1,January,09,9,1,37,7,9
10,201501BS70011,527590.0,178660.0,-0.163542,51.492497,1,3,2,1,2015-01-18,...,1,2015,1,January,15,9,2,85,11,12


In [43]:
accidents_data.shape

(429239, 38)

In [44]:
vehicles_data.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_Home_Area_Type
0,201506E098757,2,9,0,18,0,8,0,0,0,...,3,1,6,1,45,7,1794,1,11,1
1,201506E098766,1,9,0,9,0,8,0,0,0,...,4,1,6,2,25,5,1582,2,1,NaN
2,201506E098766,2,9,0,18,0,8,0,0,0,...,1,1,6,1,51,8,NaN,NaN,NaN,1
3,201506E098777,1,20,0,4,0,0,0,0,0,...,1,1,1,1,50,8,4462,2,1,1
4,201506E098780,1,9,0,15,0,1,0,0,0,...,4,1,6,1,27,6,1598,2,NaN,1


In [45]:
vehicles_data.shape

(1079601, 21)

In [46]:
casualties_data.head()

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Casualty_Type,Casualty_Home_Area_Type
0,201597UA71710,2,1,1,2,75,10,3,0,0,0,0,9,3
1,201597UA71810,2,1,2,2,63,9,2,0,0,0,4,11,3
2,201597UA71810,2,2,2,2,75,10,2,0,0,0,4,11,1
3,201597UA71810,2,3,2,1,78,11,2,0,0,0,4,11,1
4,201597UA71810,2,4,2,1,67,10,2,0,0,0,4,11,1


In [47]:
casualties_data.shape

(773502, 14)

In [52]:
# Sevirety of accident:  

x = accidents_data.Accident_Severity.groupby([accidents_data.Year]).count().index

trace_3 = go.Bar(
            x=x,
            y=accidents_data[accidents_data.Accident_Severity == 'Fatal'].Accident_Severity.groupby([accidents_data.Year]).count(),
            name='Fatal',
            marker=dict(
            color="red"))

trace_2 = go.Bar(
            x=x,
            y=accidents_data[accidents_data.Accident_Severity == 'Serious'].Accident_Severity.groupby([accidents_data.Year]).count(),
            name='Serious',
            marker=dict(
            color="green"))

trace_1 = go.Bar(
            x=x,
            y=accidents_data[accidents_data.Accident_Severity == 'Slight'].Accident_Severity.groupby([accidents_data.Year]).count(),
            name='Slight',
            marker=dict(
            color="blue"))

trace_4 = go.Scatter(
    x = x,
    y = accidents_data.Accident_Severity.groupby([accidents_data.Year]).count(), 
    name = 'Total per year', 
    mode = 'lines',
    legendgroup= 'group2', 
    marker=dict(
            color="magenta")
)

layout = go.Layout(title="Accidents Severity Per Year", 
                        barmode='group', 
                        xaxis = dict(ticks='', nticks=24, 
                                    title=go.layout.xaxis.Title(
                                    text='Year')),
                        yaxis = dict(title=go.layout.yaxis.Title(
                                    text='Count')))

fig = go.Figure(data=[trace_1, trace_2, trace_3, trace_4], layout=layout)
py.iplot(fig, filename='Accidents_Severity_Per_Year')

PlotlyRequestError: Authentication credentials were not provided.